# Отбор признаков



In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


## Данные

Мы будем работать с данными из соревнования Home Credit Default Risk в котором требовалось предсказать вернет ли клиент кредит

https://www.kaggle.com/c/home-credit-default-risk

Загрузим данные и посмотрим на них

In [2]:
application_train = pd.read_csv('application_train.csv')
application_train.shape

(307511, 122)

In [3]:
application_train.sample(5)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
189693,"319945,1,Cash loans,M,N,Y,0,112500.0,521280.0,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237238,374784,0.0,Cash loans,F,Y,Y,0.0,225000.0,814041.0,23931.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
27793,"132312,0,Cash loans,F,Y,Y,1,180000.0,980005.5,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121667,241059,0.0,Cash loans,F,Y,Y,1.0,180000.0,144000.0,9333.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
211289,344848,0.0,Cash loans,F,N,N,0.0,112500.0,508495.5,35518.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
application_train.TARGET.value_counts() # число уникальных строк

0.0    214121
1.0     19278
Name: TARGET, dtype: int64

Для удобства далее будем рассматриватьлишь 10% данных.


In [7]:
from sklearn.model_selection import train_test_split

application, _ = train_test_split(application_train,
                                  train_size=0.1,
                                  random_state=27,
                                  stratify=application_train.TARGET #Для сохранения баланса классов (относительно target)
                                  )
application = application.sort_values('SK_ID_CURR').reset_index(drop=True)
application.TARGET.value_counts()

ValueError: ignored

In [ ]:
application.head()

application_train довольно большая таблица, дальше она нам не нужна, можно ее удалить и собрать мусор

In [ ]:
del application_train
gc.collect(); #Собираем мусор

Выделим числовые и нечисловые признаки

In [ ]:
categorical_list = []
numerical_list = []
for i in application.columns.tolist():
    if application[i].dtype=='object':
        categorical_list.append(i)
    else:
        numerical_list.append(i)
print('Number of categorical features:', len(categorical_list))
print('Number of numerical features:', len(numerical_list))

Посмотрим на наличие пропущенных значений

In [ ]:
application.isnull().sum()

Для замены пропущенных значений можно выспользоваться `SimpleImputer` из sklearn: данная модель заменяет пропущенные значения (`np.nan`) каким-то образом `strategy` (по умолчанию заменяет средним, но можно и медиану, самым частым значением или указанной в `fill_value` константой)

In [ ]:
from sklearn.impute import SimpleImputer
application[numerical_list] = SimpleImputer(missing_values=np.nan,
                                            strategy='median').fit_transform(application[numerical_list])

In [ ]:
application.isnull().sum()

In [ ]:
application.isnull().sum().any()

Теперь все числовые данные не имеют пропусков, но некоторые категориальные - да. Разберемся с ними

In [ ]:
application = pd.get_dummies(application, drop_first=True)
print(application.shape)

In [ ]:
application.isnull().sum().any()

In [ ]:
application.head()

In [ ]:
application.info()

Теперь выделим таргет (TARGET) и удалим SK_ID_CURR (Вопрос: Почему удаляем данный признак?)

In [ ]:
X = application.drop(['SK_ID_CURR', 'TARGET'], axis=1) # ID - уникальные. Ни о чем не говорят. Это способ переобучения. Нахрен он нам нужен?
y = application.TARGET
feature_name = X.columns.tolist()

In [ ]:
application.drop['SK_ID_CURR'].nunique()

In [ ]:
X.shape

Теперь есть 224 признака, будем пробовать выбрать лучшие

## Одномерные методы

Идея: оценить важность каждого признака по отдельности, выбрать самые важные признаки

In [ ]:
def feature_selector(X, y, score_function, n_features=100):
    importance_list = []
    feature_names = X.columns.to_numpy()
    # Считаем важность для каждого признака
    for i in feature_names:
        importance_list.append(score_function(X[i], y))
    # Заменяем np.nan на 0
    importance_list = [0 if np.isnan(i) else i for i in importance_list]
    # Выбрали названия признаков с наибольшей важностью
    best_features = feature_names[np.argsort(importance_list)[-n_features:]][::-1]

    return best_features

### Корреляция Пирсона

Идея: подсчитали корреляцию признака $x^j$ и таргета ($R(x^j, y)$), если корреляция большая по модулю, значит признак информативный


$$R(x, y) = \frac{\sum_{i=1}^n(x_i - \overline{x})(y_i - \overline{y})}{\sqrt{\sum_{i=1}^n(x_i - \overline{x})^2 \sum_{i=1}^n(y_i - \overline{y})^2}}$$



In [ ]:
def pearson_correlation_abs(x, y):
  return np.abs(np.corrcoef(x, y)[0, 1]) # отриц. зав-ть - тоже зав-ть

In [ ]:
cor_features = feature_selector(X, y, score_function=pearson_correlation_abs)
print(str(len(cor_features)), 'selected features')

In [ ]:
fig, axs = plt.subplots(figsize=(10,20), nrows=4, ncols=2)

for i in range(4):
  sns.boxplot(y=X[cor_features[i]],
               x=y,
               ax=axs[i][0])
  sns.histplot(x=X[cor_features[i]],
               hue=y,
               ax=axs[i][1])

Проблема данного подхода: учитывается только линейная связь

### 2. T score (для бин. кл-ии)

Идея: подсчитали t score признака $x^j$ на основе разделения по таргету таргета ($R(x^j, y)$), если t score большой, значит признак информативный


$$R(x, y) = \frac{|\mu_1 - \mu_0|}{\sqrt{\frac{\sigma_0^2}{n_0}+\frac{\sigma_1^2}{n_1}}}$$, где

$\mu_i, \sigma_i^2, n_i$ - это среднее, дисперсия и количество объектов для признака $x$ класса $i$ (0 или 1)

Данный метод используется для задачи бинарной классификации (для многоклассовой есть F score)

In [ ]:
def t_score(x, y):
  def calc_stats(x):
    return np.mean(x), np.var(x), len(x)

  mu0, s0, n0 = calc_stats(x[y == 0.0])
  mu1, s1, n1 = calc_stats(x[y == 1.0])
  return np.abs(mu1-mu0) / np.sqrt(s0/n0 + s1/n1)


In [ ]:
tscore_features = feature_selector(X, y, score_function=t_score)
print(str(len(tscore_features)), 'selected features')

In [ ]:
fig, axs = plt.subplots(figsize=(10,20), nrows=4, ncols=2)

for i in range(4):
  sns.boxplot(y=X[tscore_features[i]],
               x=y,
               ax=axs[i][0])
  sns.histplot(x=X[tscore_features[i]],
               hue=y,
               ax=axs[i][1])

In [ ]:
set(cor_features) == set(tscore_features)

Основаня проблема одномерных методов - не работают, если целевая переменная зависит от совокупности признаков

## Методы обертки (Wrapper)

Идея:  оценить поднаборы признков, делая возможным обнаружения возможную взаимосвязь между совокупностью признаков

*RFE*(Recursive Feature Elimination) - обучается на начальном наборе признаков, и важность каждого признака получается либо через атрибут `coef_`, либо через атрибут `feature_importances_` модели, указанной в `estimator`.

Затем `step` наименее важных признаков удаляются. Эту процедура рекурсивно повторяется, пока в конечном итоге не будет достигнуто `n_features_to_select` признаков.

In [ ]:
from sklearn.preprocessing import StandardScaler
X_norm = StandardScaler().fit_transform(X)

In [ ]:
lr = LogisticRegression().fit(X, y)
lr.coef_ # Нихрена не отнормировано. Не можем судить о важности признака

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=100, step=10, verbose=5)
rfe_selector.fit(X_norm, y)

In [ ]:
rfe_support = rfe_selector.get_support() # Получаем маску True/False для признаков
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

Основная проблема - вычислительно дорого

## Встроенные методы (Embeded)

Идея `SelectFromModel`: через `estimator` подсчитывается важность признаков. Если важность меньше порогового значения - признак убирается. Пороговое значение задается параметром `threshold` -можно задать числом или указать эвристику: “mean”, “median”, дополнительно можно добавить дробь (“0.1*mean”)


In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(estimator=LogisticRegression(penalty="l2"), threshold='1.25*median')
embeded_lr_selector.fit(X_norm, y)

In [ ]:
embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = X.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

### Random Forest

Для некоторых моделей важность признаков - это атрибут `coef_` (Вопрос: можете привести примеры? Лог. рег., лин. рег.), но у леса такого атрибута нет (Вопрос: почему? потому что нет весов)


У леса есть атрибут `feature_importances_` - важность признака подсчитывается как нормализованная сумма уменьшений критерия по всем деревьям, по всем вершинам, где было разбиение по данному признаку.

Уменьшение критерия = $H(X_m) - \frac{|X_l|}{|X_m|} H(X_l) - \frac{|X_r|}{|X_m|} H(X_r)$



In [8]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X, y)

NameError: ignored

In [ ]:
plot = sns.barplot(y=feature_name,
                   x=clf.feature_importances_,
                   order=np.array(feature_name)[np.argsort(clf.feature_importances_)][::-1]
                   )
plot.figure.set_size_inches(10, 50)

Применим `SelectFromModel` к `RandomForestClassifier`

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100),
                                      threshold='1.25*median')
embeded_rf_selector.fit(X, y)

In [ ]:
embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = X.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

### LGBMClassifier

`SelectFromModel` можно использовать не только с моделями из sklearn, например, можно использовать `LGBMClassifier` (у него тоже есть `feature_importances_`)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
import re
X_renamed = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

lgbc=LGBMClassifier(n_estimators=500,
                    learning_rate=0.05,
                    num_leaves=32,
                    colsample_bytree=0.2,
                    reg_alpha=3,
                    reg_lambda=1,
                    min_split_gain=0.01,
                    min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, threshold='1.25*median')
embeded_lgb_selector.fit(X_renamed, y)

In [ ]:
embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = X.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')